# Retrieval Augmented Generation(RAG) 사용해 보기
- 현재 LLM은 `Fine-tuning(특히 Low Rank Adaptation)`과 `Retrieval Augmented Generation(RAG)`라는 방식으로 발전하고 있음
- `Fine-tuning`
    - 모델의 가중치를 갱신하기 위해 데이터를 추가적으로 학습시키는 방식으로, 상대적으로 오랜 시간이 걸림
    - `시험을 치르기 위해 모델에게 공부를 시키는 것`
- `RAG`
    - LLM은 태생적으로 Hallucination(환각) 증상을 가지고 있음
    - 이를 해결하기 위해, LLM이 답변을 위해 필요한 맥락 정보를 같이 전달하여 이를 예방하고 좀 더 원하는 결과를 얻을 수 있음
    - `오픈북 시험`

## 작동 방식
1. RAG에 사용할 데이터를 추출
2. 데이터를 구조화/청킹하여 메타데이터와 함게 임베딩 및 벡터DB에 저장
3. 사용자의 질의를 임베딩
4. 벡터DB에서 사용자 질의 임베딩과 유사한 벡터를 찾고 해당 데이터를 추출 이를 문맥 정보로 활용
5. LLM에 문맥과 사용자 질의를 전달하여 답변시 좀 더 활용할 수 있도록 함

![](img/1.png)
![](img/2.png)

## PDF 내용을 기준으로 답변해 주는 챗봇 만들어 보기

## 사용할 모델 정의

### 임베딩: HuggingFaceEmbeddings

### LLM: 42dot

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import LlamaCpp

In [2]:
EMBEDDING_MODEL_NAME = "jhgan/ko-sroberta-multitask"
LLM_MODEL_PATH = 'models/42dot_LLM-SFT-1.3B_Q4_K_M.gguf'

## VetorDB 정의
- 다양한 DB가 있으나, 예제에서는 `faiss`를 사용함

In [3]:
from langchain_community.vectorstores import FAISS  # pip install faiss-cpu 필요

## VectorDB 데이터 생성
1. RAG에 사용할 데이터 로드(여기서는 PDF)
2. 적절한 형태로 분리
3. 각 청크를 임베딩 모델을 이용하여 임베딩 및 VectorDB에 저장

## 데이터 로드
- 의미있는 형태로만 가져올 수 있다면 어떤 데이터든 벡터화할 수 있음
- 예제에서는 PDF파일(data/개인정보_보호_가이드라인.pdf)의 내용을 가져오는 PDF 파서를 이용함 

In [4]:
DB_NAME = "faiss_index_st"  # 데이터가 저장될/된 곳
FILE_PATH = "data/개인정보_보호_가이드라인.pdf"

# pdf 파서를 이용해 데이터를 읽고 분리
# PyPDFLoader는 pypdf라는 라이브러리는 래핑하여 langchain에서 쉽게 사용할 수 있도록 해줌 (pip install pypdf 필요)
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(FILE_PATH)
data_list = loader.load()

In [5]:
data_list[4]

Document(page_content='Personal Information Protection Guidelines  • 5\n  신기술 환경에 부합하는 개인정보 처리기준 제시로 법 실효성 확보\n•     AI, 영상·생체인식정보·위치정보 처리기기 등 신기술 장치 운영 시 안전한 개인정보 처리·보호 \n방법·절차 등을 제시\n•     재택근무, 정보유출 방지 등을 위한 개인정보 처리 시 준수사항 안내\n  침해사고 예방 및 정보주체 권리 보장\n•     사업자의 업무 효율과 개인정보 보호에 대한 사회적 관심 증가를 고려하여, 근로자 개인정보 \n침해사고 예방방안 제시\n•     개인정보 침해사고 발생 등에 따른 침해신고, 분쟁해결, 손해배상 청구 등 정보주체 권리보장  \n방법 안내\n2       적용 대상 및 범위\n  적용대상\n•     입사 지원자 및 근로자\n•     근로자 등의 개인정보를 처리하는 사용자\n  적용범위 : 채용 준비부터 고용 종료까지 인사·노무 업무 전 과정\n①  채용 준비 : 채용 전형, 합격자 통보 등과 관련한 개인정보 처리·보호 방법\n②  채용 결정 : 근로자명부 작성 등 법령 준수, 근로계약의 체결·이행 등에 따른 개인정보 처리·보호\n③  고용 유지 :   인사이동, 보수·복리후생 등 인력관리에 따른 개인정보 처리·보호 방법 및 디지털 장치 \n도입 시 준수사항 등\n④  고용 종료 :   퇴직 근로자의 개인정보 파기 및 경력증명서 발급 등에 필요한 개인정보 처리·보호 방법', metadata={'source': 'data/개인정보_보호_가이드라인.pdf', 'page': 4})

## 데이터 분리
- PDF 파서를 통해 가져온 데이터는 한 단락 혹은 페이지 별로 데이터를 가지고 있음
- 너무 많은 정보를 하나의 벡터로 표현하는 것은 각 벡터간의 의미가 퇴색되어, 적절하지 않을 수 있음
- 이를 위해 여러가지 방법이 존재하며, 본 예제에서는 가장 간단한 형태인 ChracterTextSplitter를 사용하여 특정 크기별로 데이터를 분리함
- chunk_overlap을 통해 각 벡터들간의 연관성을 조금씩 두고자 함

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 300글자 단위로 쪼갬
    chunk_overlap=30,  # 30글자를 겹치게 함
    length_function=len,  # 청크의 길이 기준이 되는 함수
    is_separator_regex=False,
)

# data_list는 langchain이 사용할 수 있게 래핑된 PyPDFLoader를 사용했기 때문에 langchain 내부에서 사용하는 Document라는 객체 단위로 구분됨
# RecursiveCharacterTextSplitter 역시 List[Document]를 받아 List[Document]를 반환함
texts = text_splitter.split_documents(data_list)

In [7]:
texts[8]

Document(page_content='Personal Information Protection Guidelines  • 5\n  신기술 환경에 부합하는 개인정보 처리기준 제시로 법 실효성 확보\n•     AI, 영상·생체인식정보·위치정보 처리기기 등 신기술 장치 운영 시 안전한 개인정보 처리·보호 \n방법·절차 등을 제시\n•     재택근무, 정보유출 방지 등을 위한 개인정보 처리 시 준수사항 안내\n  침해사고 예방 및 정보주체 권리 보장\n•     사업자의 업무 효율과 개인정보 보호에 대한 사회적 관심 증가를 고려하여, 근로자 개인정보 \n침해사고 예방방안 제시', metadata={'source': 'data/개인정보_보호_가이드라인.pdf', 'page': 4})

In [8]:
texts[9]

Document(page_content='침해사고 예방방안 제시\n•     개인정보 침해사고 발생 등에 따른 침해신고, 분쟁해결, 손해배상 청구 등 정보주체 권리보장  \n방법 안내\n2       적용 대상 및 범위\n  적용대상\n•     입사 지원자 및 근로자\n•     근로자 등의 개인정보를 처리하는 사용자\n  적용범위 : 채용 준비부터 고용 종료까지 인사·노무 업무 전 과정\n①  채용 준비 : 채용 전형, 합격자 통보 등과 관련한 개인정보 처리·보호 방법\n②  채용 결정 : 근로자명부 작성 등 법령 준수, 근로계약의 체결·이행 등에 따른 개인정보 처리·보호', metadata={'source': 'data/개인정보_보호_가이드라인.pdf', 'page': 4})

## 임베딩 모델 지정

In [9]:
embedding_model = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    cache_folder="models",
)

/Users/sucream/python/ai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 임베딩 작업 및 저장
- FAISS, HuggingFaceEmbeddings도 langchain에서 사용하기 좋게 래핑되었기 때문에 쉽게 임베딩 및 db에 저장할 수 있음

In [10]:
# FAISS VectorDB에 texts 내에 있는 Document를 하나씩 embedding_model을 이용하여 임베딩하고 저장
faiss_index = FAISS.from_documents(texts, embedding_model)
# 아직까지 VectorDB는 메모리에만 존재함
# 이를 위해 save_local 함수로 저장할 수 있음
faiss_index.save_local(DB_NAME)

## VectorDB 로드
- 로컬에 저장된 DB를 불러올 수 있음

In [11]:
# FAISS를 불러오는데, 향후 이 vectorDB에 임베딩으로 사용할 모델을 같이 넣어줘야 함
faiss_index = FAISS.load_local(DB_NAME, embedding_model)

## 질의

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [13]:
# 해당 vectorDB를 retriever로 사용
# k: 3은 사용자 질의와 가장 유사한 데이터 3개를 문맥으로 같이 넘기라는 의미
retriever = faiss_index.as_retriever(search_kwargs={"k": 3})

In [15]:
# 모델 생성
llm = LlamaCpp(
    model_path=LLM_MODEL_PATH,
    temperature=0,
    top_p=0.98,
    top_k=0,
    n_ctx=32768,
    max_tokens=1024,
    repeat_penalty=1.176,
    last_n_tokens_size=128,
    verbose=False,
    stop=["</s>"],
)

llama_model_loader: loaded meta data with 23 key-value pairs and 219 tensors from models/42dot_LLM-SFT-1.3B_Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 24
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.head_count u3

In [16]:
# LLM에 전달할 프롬프트 생성
# LLM이 문맥을 이용할 수 있도록 context를 하나 더 만듦

template = """\
아래는 대답을 위한 문맥 정보입니다.
---------------------
{context}
---------------------
주어진 문맥 정보와 사전 지식을 활용하여, 다음 질문에 대한 답변을 해주세요.
만약 답변을 찾을 수 없다면, '죄송합니다. 잘 모르겠어요.'라고 답변하세요.

질문: {question}
답변: \
"""

prompt = PromptTemplate.from_template(template)

In [17]:
# 실제 사용할 최종 모델
retrievalQA = RetrievalQA.from_llm(llm, prompt, retriever=retriever)

In [20]:
query = input("질문: ")
print(retrievalQA.invoke(query)["result"])

질문:  근로자의 가족 개인정보를 확인한 후 적절한 복지혜택을 제공하고자 합니다. 이 경우에 근로자 가족의 동의를 받아야 하나요?


 가족수당의 기초가 되거나 가족 복지혜택 제공을 위해 필요한 개인정보의 수집은 근로자 또는 가족의 동의 없이도 가능합니다. 다만 고유식별정보나 민감정보를 수집하는 경우에는 별도의 동의를 받도록 해야 하며 주민등록번호는 관련 법령에 따라 처리해야 합니다.
